In [3]:
import os
import sys
import subprocess
import shutil
import io

# --- 1. Xoá thư mục chromadb cũ nếu tồn tại ---
if os.path.isdir("chromadb"):
    print("Đang xóa thư mục chromadb cũ...")
    shutil.rmtree("chromadb")

# --- 3. Kiểm tra và cài đặt các thư viện cần thiết ---
def ensure_package(pkg_name, import_name=None):
    try:
        __import__(import_name or pkg_name)
    except ImportError:
        print(f"Thiếu thư viện '{pkg_name}', đang cài đặt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        __import__(import_name or pkg_name)

pkgs = [
    ("chromadb", None),
    ("langchain", None),
    ("ollama", None),
    ("tiktoken", None),
    ("PyPDF2", None)
]

for pkg, imp in pkgs:
    ensure_package(pkg, imp)

Thiếu thư viện 'chromadb', đang cài đặt...
Thiếu thư viện 'langchain', đang cài đặt...
Thiếu thư viện 'ollama', đang cài đặt...
Thiếu thư viện 'tiktoken', đang cài đặt...
Thiếu thư viện 'PyPDF2', đang cài đặt...


In [ ]:
# --- Đọc PDF với decoder UTF-8 để tránh lỗi tuple ---
import os
from PyPDF2 import PdfReader
from langchain.schema import Document

pdf_path = "./your_file.pdf"  # Thay đường dẫn tới file PDF của bạn
reader = PdfReader(pdf_path)
docs = []

for i, page in enumerate(reader.pages):
    raw_text = page.extract_text() or ""
    # Nếu raw_text là bytes, decode bằng utf-8 và bỏ ký tự không hợp lệ
    if isinstance(raw_text, (bytes, bytearray)):
        text = raw_text.decode("utf-8", errors="ignore")
    else:
        text = raw_text
    docs.append(Document(
        page_content=text,
        metadata={"source": f"{os.path.basename(pdf_path)}_page_{i+1}"}
    ))

In [ ]:
# --- 4. Import sau khi đã đảm bảo ---
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

# --- 5. Load PDF và chuẩn bị embedding + vector store local ---
pdf_path = "./your_file.pdf" # chỉnh đường dẫn theo file của bạn
docs = PyPDFLoader(pdf_path).load()

In [ ]:
# Sử dụng embedding model local
embed_model = OllamaEmbeddings(model="gpt-oss:20b")

In [ ]:
# Tạo hoặc tải lại vector database local (chromadb)
vectordb = Chroma.from_documents(
documents=docs,
embedding=embed_model,
persist_directory="chromadb"
)
# Lưu persist để tái sử dụng lần tới
vectordb.persist()

In [ ]:
# --- 6. Khởi tạo LLM local cho trả lời ---
llm = Ollama(model="gpt-oss:20b") # hoặc llama3:latest

In [ ]:
# --- 7. Thiết lập pipeline RAG ---
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever=retriever,
return_source_documents=True
)

In [ ]:
# --- 8. Hàm hỏi đáp ---
def ask(question: str):
result = qa_chain({'query': question})
print("=== ANSWER ===")
print(result['result'])
print("=== SOURCES ===")
for src in result['source_documents']:
print(f"- {os.path.basename(src.metadata.get('source', ''))}")

In [ ]:
# Ví dụ sử dụng
aif __name__ == "__main__":
ask("Nhập câu hỏi của bạn về nội dung PDF ở đây...")